In [1]:
import sys
sys.path.append('../src') 

# Importing libraries
import pandas as pd
import numpy as np

# Libraries for machine learning
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

# Importing script
import etl as etl

import warnings
warnings.filterwarnings('ignore')

## Disease Risk Classification Model

The purpose of this notebook is to explore a variety of models and tune them in order to figure out which is best for disease risk classification. According to the article "Comparing different supervised machine learning algorithms for disease prediction" by Shahadat Uddin et al. (https://bmcmedinformdecismak.biomedcentral.com/articles/10.1186/s12911-019-1004-8), Support Vector Machines, Naive Bayes, and Random Forest are some of the most common machine learning algorithms applied to disease prediction. We will explore these along with other algorithms such as Logistic Regression and K Nearest Neighbors in this notebook.

Let's get straight into it and simulate a data set of 5,000 individuals.

In [2]:
simulated_gwas_fp = '../testdata/gwas/gwas_simulate.tsv' 
maf_fp = '../references/snp_mafs.txt'
simulated_data = etl.simulate_data(simulated_gwas_fp, maf_fp, 5000)
simulated_data.head()

,rs1692580,rs2843152,rs36096196,rs10909862,rs2493298,rs17037390,rs10127456,rs35465346,rs113716316,rs12023377,...,rs111245230,rs10981012,rs41311933,rs10818576,rs885150,rs579459,rs495828,rs16999497,PRS,Class
0,0,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,226.012239,1
1,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,86.663000,0
2,0,1,0,1,0,0,0,0,0,0,...,0,0,0,1,1,1,0,1,86.497028,0
3,0,1,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,91.766276,0
4,1,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,137.421000,0


We will not be using the simulated data above for building the model. Given that the class label (disease risk category) was assigned to an individual depending on the weighted sum of all and only the SNPs in this data set, it would be too easy for a machine learning model to figure this out. Essentially, the above data set is a simulated 'ground truth'. In machine learning problems you typically don't work with all the variables that determine the label so something needs to be done about this.

As a solution, we will be making classifications based on a subset of SNPs in the above data set. To do this, we will be using a separate GWAS to inform us of SNPs that are most important in predicting disease. Only those SNPs that are in the above data set and the new GWAS will be used. The model will then be trained on this subset.

Let's load in the other GWAS data and filter the above data.

In [3]:
model_gwas_fp = '../testdata/gwas/gwas_model.tsv' 
model_data = pd.read_csv(model_gwas_fp, sep='\t')
subset = set(simulated_data.columns).intersection(model_data['SNPS'].unique())
new_columns = list(subset)+['Class']

data = simulated_data[new_columns]

Now let's create a training and test set on the above data.

In [4]:
X = data.drop('Class', axis=1)
y = data['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=10)

Now we can train different models on these sets and assess their results.

**Note:**

>It is important to note that accuracy is not the only metric that is important here to assess the performance of the models. Since we are working with predicting the disease risk of individuals, **Type I** and **Type II** errors are also very important. It is potentially dangerous to classify an individual as low risk when they are actually high risk (Type II, False Negative). Additionally, classifying an individual as high risk when they are actually low risk could cause the individual some unecessary stress within themselves and their families (Type I, False Positive). Therefore, it is important to control for such errors in our model. To do so, we will prioritize the maximization of **Recall** (TP/TP+FN) since it is an indicator of the dangerous False Negatives in our model but at the same time attempt to maximize **Precision** (TP/TP+FP) since it is an indicator of False Positives. 

### Logistic Regression

In [5]:
lg = LogisticRegression()
lg.fit(X_train, y_train)

accuracy_lg = lg.score(X_test, y_test)*100
print('The accuracy for the Logistic Regression model is {}%'.format(accuracy_lg))

The accuracy for the Logistic Regression model is 85.28%


Let's attempt to improve the model using grid search.

In [6]:
lg_parameters = {'tol':[.1, .001, .0001], 'C':[10, 1, .1]}
lg = LogisticRegression()
clf1 = GridSearchCV(lg, lg_parameters)

clf1.fit(X_train, y_train)
preds_lg = clf1.predict(X_test)
accuracy_lr = np.mean(y_test == preds_lg)*100


print('The accuracy for the refined Logistic Regression model is {}%'.format(accuracy_lr))

The accuracy for the refined Logistic Regression model is 90.56%


In [7]:
target_names = ['Low Risk', 'Medium Risk', 'High Risk']
print(classification_report(y_test, preds_lg, target_names=target_names))

              precision    recall  f1-score   support

    Low Risk       0.98      1.00      0.99       693
 Medium Risk       0.88      0.79      0.83       367
   High Risk       0.69      0.79      0.74       190

    accuracy                           0.91      1250
   macro avg       0.85      0.86      0.85      1250
weighted avg       0.91      0.91      0.91      1250



### K Nearest Neighbors

In [8]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

accuracy_knn = knn.score(X_test, y_test)
print('The accuracy for the K Nearest Neighbors model is {}%'.format(accuracy_knn))

The accuracy for the K Nearest Neighbors model is 0.7712%


Let's attempt to improve the model using grid search.

In [9]:
knn_parameters = {'n_neighbors':[10, 5, 3, 1], 'p':[2, 1]}
knn = KNeighborsClassifier()
clf2 = GridSearchCV(knn, knn_parameters)

clf2.fit(X_train, y_train)
preds_knn = clf2.predict(X_test)
accuracy_knn = np.mean(y_test == preds_knn)*100

print('The accuracy for the refined K Nearest Neighbors model is {}%'.format(accuracy_knn))

The accuracy for the refined K Nearest Neighbors model is 79.67999999999999%


In [10]:
print(classification_report(y_test, preds_knn, target_names=target_names))

              precision    recall  f1-score   support

    Low Risk       0.76      1.00      0.86       693
 Medium Risk       1.00      0.31      0.47       367
   High Risk       0.84      1.00      0.91       190

    accuracy                           0.80      1250
   macro avg       0.87      0.77      0.75      1250
weighted avg       0.84      0.80      0.76      1250



### Support Vector Machine

In [11]:
svc = SVC()
svc.fit(X_train, y_train)

accuracy_svc = svc.score(X_test, y_test)*100
print('The accuracy for the Support Vector Machine model is {}%'.format(accuracy_svc))

The accuracy for the Support Vector Machine model is 99.92%


In [13]:
svc_parameters = {'tol': [.1, .001, .0001], 'C':[10, 1, .1]}
svc = SVC()
clf3 = GridSearchCV(svc, svc_parameters)

clf3.fit(X_train, y_train)
preds_svc = clf3.predict(X_test)
accuracy_svc = np.mean(y_test == preds_svc)

print('The accuracy for the refined Support Vector Machine model is {}'.format(accuracy_svc))

The accuracy for the refined Support Vector Machine model is 0.9992


In [14]:
print(classification_report(y_test, preds_svc, target_names=target_names))

              precision    recall  f1-score   support

    Low Risk       1.00      1.00      1.00       693
 Medium Risk       1.00      1.00      1.00       367
   High Risk       1.00      1.00      1.00       190

    accuracy                           1.00      1250
   macro avg       1.00      1.00      1.00      1250
weighted avg       1.00      1.00      1.00      1250



### Naive Bayes

In [15]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)

accuracy_gnb = gnb.score(X_test, y_test)*100
print('The accuracy for the Naive Bayes is {}%'.format(accuracy_gnb))

The accuracy for the Naive Bayes is 95.76%


In [16]:
gnb_parameters = {'var_smoothing':[1e-3, 1e-6, 1e-9]}
gnb = GaussianNB()
clf4 = GridSearchCV(gnb, gnb_parameters)

clf4.fit(X_train, y_train)
preds_gnb = clf4.predict(X_test)
accuracy_gnb = np.mean(y_test == preds_gnb)

print('The accuracy for the refined Naive Bayes model is {}'.format(accuracy_gnb))

The accuracy for the refined Naive Bayes model is 0.9696


In [17]:
print(classification_report(y_test, preds_gnb, target_names=target_names))

              precision    recall  f1-score   support

    Low Risk       1.00      0.96      0.98       693
 Medium Risk       0.91      0.99      0.95       367
   High Risk       1.00      0.95      0.98       190

    accuracy                           0.97      1250
   macro avg       0.97      0.97      0.97      1250
weighted avg       0.97      0.97      0.97      1250



### Random Forest

In [18]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

accuracy_rf = rf.score(X_test, y_test)*100
print('The accuracy for the Random Forest is {}%'.format(accuracy_rf))

The accuracy for the Random Forest is 81.67999999999999%


In [19]:
rf_parameters = {'n_estimators':[100, 50, 10]}
rf = RandomForestClassifier()
clf5 = GridSearchCV(rf, rf_parameters)

clf5.fit(X_train, y_train)
preds_rf = clf5.predict(X_test)
accuracy_rf = np.mean(y_test == preds_rf)

print('The accuracy for the refined Random Forest model is {}'.format(accuracy_rf))

The accuracy for the refined Random Forest model is 0.9264


In [20]:
print(classification_report(y_test, preds_rf, target_names=target_names))

              precision    recall  f1-score   support

    Low Risk       0.89      1.00      0.94       693
 Medium Risk       0.99      0.75      0.86       367
   High Risk       1.00      0.99      0.99       190

    accuracy                           0.93      1250
   macro avg       0.96      0.91      0.93      1250
weighted avg       0.93      0.93      0.92      1250

